# SSQM v2.0 Comparing two seasons
## Using NBA Shooting Data: Considers shot type, closest defender distance and touch time
## Bin shots by filtering each condition combination and then use that for SSQM

In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

player_dict = get_players_pbp()
teams_dict, teams_list = get_teams()

data_DIR1 = "../data/box/"
data_DIR2 = "../data/shots/"
csv_export_DIR = "C:/Users/pansr/Documents/repos/csv/"
img_DIR_P = "../data/images/players/"
fig_DIR = "../figs/shots/"

In [ ]:
def get_ssqm(season):
    dft = pd.read_parquet(data_DIR1 + f"NBA_Box_P_Cum_Base_{season}.parquet", columns = ["PLAYER_ID","TEAM_ID"])
    df = pd.read_parquet(data_DIR2 + f"NBA_Shots_{season}_All.parquet")
    dfd = pd.read_parquet(data_DIR2 + f"NBA_Shots_{season}_overall.parquet", columns = ["PLAYER_ID","FGM","FGA"])
    dfd = dfd.sort_values(by = "PLAYER_ID").reset_index(drop=True)

    df = df[["PLAYER_ID","PLAYER_NAME","PLAYER_LAST_TEAM_ID","FGM","FGA","FG2M","FG2A","FG3M","FG3A", 'general_range', 'closest_def', 'touch_time']]
    df = df.query("general_range != 'Other'")
    df_avg = df.groupby(['general_range', 'closest_def', 'touch_time']).sum()
    df_avg = df_avg.drop(columns= ["PLAYER_ID","PLAYER_NAME","PLAYER_LAST_TEAM_ID"])
    df_avg["xFG2"] = df_avg["FG2M"]/df_avg["FG2A"]
    df_avg["xFG3"] = df_avg["FG3M"]/df_avg["FG3A"]
    df_avg = df_avg.drop(columns =["FGM","FGA","FG2M","FG2A","FG3M","FG3A"])
    df_avg = df_avg.reset_index()

    shots = pd.merge(df,df_avg,on=['general_range', 'closest_def', 'touch_time'])
    shots["FG2_PCT"] = shots["FG2M"]/shots["FG2A"]
    shots["FG3_PCT"] = shots["FG3M"]/shots["FG3A"]
    shots = shots.replace([np.inf, -np.inf], np.nan)
    shots = shots.fillna(0)
    shots["PTS2"] =  (2*shots["FG2A"]*shots["FG2_PCT"]).round(2)
    shots["PTS3"] =  (3*shots["FG3A"]*shots["FG3_PCT"]).round(2)
    shots["PTS"] =  (2*shots["FG2A"]*shots["FG2_PCT"] + 3*shots["FG3A"]*shots["FG3_PCT"]).round(2)
    shots["xPTS2"] = (2*shots["FG2A"]*shots["xFG2"]).round(2)
    shots["xPTS3"] = (3*shots["FG3A"]*shots["xFG3"]).round(2)
    shots["xPTS"] = (2*shots["FG2A"]*shots["xFG2"] + 3*shots["FG3A"]*shots["xFG3"]).round(2)

    fg = (shots
        .groupby(['PLAYER_ID'])[['FGM', 'FGA', 'PTS', 'xPTS']]
        .agg({'FGM': ["sum"], 'FGA': ["sum"], 'PTS': ["sum"], 'xPTS': ["sum"]}))
    fg.columns = ['FGM', 'FGA', 'PTS', 'xPTS']
    fg['eFG'] = np.round(fg['PTS']/fg['FGA']/2, 3)
    fg['xeFG'] = np.round(fg['xPTS']/fg['FGA']/2, 3)
    fg['Shot_Making'] = np.round((fg['PTS'] - fg['xPTS'])/fg['FGA'], 3)

    fg = fg.drop(columns=['FGM', 'FGA'])
    fg = fg.fillna(0)
    fg = pd.merge(dfd,fg,on=["PLAYER_ID"])
    fg["Points_Added"] = fg["Shot_Making"]* fg["FGA"]
    fg["PTS"] = fg["PTS"].astype(int)
    fg = fg.reset_index()
    fg["Player"] = fg["PLAYER_ID"].map(player_dict)
    fg.insert(1,"Player",fg.pop("Player"))
    fg = pd.merge(fg,dft,on="PLAYER_ID")
    fg["Team"] = fg["TEAM_ID"].map(teams_dict)
    fg.insert(2,"Team",fg.pop("Team"))
    fg[['Points_Added']] = fg[['Points_Added']].round(1)
    fg[['Shot_Making']] = fg[['Shot_Making']].round(3)
    fg = fg.drop(columns=["TEAM_ID","index"])
    return fg

In [ ]:
df1 = get_ssqm(2023)
df1["season"] = 2024
df2 = get_ssqm(2024)
df2["season"] = 2025

In [ ]:
team = "Boston Celtics"
df31 = df1.query(f"Team == '{team}'")
df32 = df2.query(f"Team == '{team}'")

In [ ]:
df3 = pd.merge(df31,df32,on=["Player","PLAYER_ID","Team"],suffixes=["1","2"])
df3["Shot_Making_diff"] = round(df3["Shot_Making2"] - df3["Shot_Making1"],3)
df3 = df3.query("PTS2 > 200")
df3 = df3.sort_values("Shot_Making_diff",ascending=False).reset_index(drop=True)

In [ ]:
df4 = df3[["Player","PLAYER_ID","Shot_Making1","Shot_Making2","Shot_Making_diff"]]
df_e = df4.reset_index()
df_e["index"] += 1
df_e["PLAYER_ID"] = df_e["PLAYER_ID"].astype(str)

In [ ]:
t = (
    GT(df_e)
    .tab_header(
        title = md(f"**{team} Difference in Shot Making**"),
        subtitle = "Between 2024-25 and 2023-24 seasons | Based on SSQM v2.0"
    )
    .tab_source_note(source_note="Shot Making: Points per shot (PPS) above league average")
    .tab_source_note(source_note = "Simple Shot Quality Model: SSQM v2.0 is based on shot type, defender distance & touch time" )
    .tab_source_note(source_note="bsky:@sradjoker.cc | X:@sradjoker | source: nba.com/stats")
    .cols_label(
        index = "#",
        PLAYER_ID = "",
        Shot_Making1 = "2024",
        Shot_Making2 = "2025",
        Shot_Making_diff = "Difference"
    )
    .data_color(
        columns=['Shot_Making1','Shot_Making2','Shot_Making_diff'],
        palette="RdBu",
        reverse="False"
    )
    .fmt_image(
        columns="PLAYER_ID",
        path = img_DIR_P,
        file_pattern="{}.png"
    )
    .cols_align(align="center")
    .cols_align(align="left", columns="Player")
    .tab_options(
        heading_title_font_size="150%",
        heading_subtitle_font_size="110%",
        # heading_title_font_weight='bold',
        table_background_color="floralwhite",
        column_labels_font_size="105%",
        column_labels_font_weight='bold',
        row_group_font_weight='bold',
        row_group_background_color="#E5E1D8",
        table_font_size=10,
        table_font_names="Consolas", 
        data_row_padding = "2px",
        # table_margin_left = 7,
        # table_margin_right = 0,
    )
)
t.save(fig_DIR + f"ssqm2_points_added_comp.png",scale=3,web_driver="firefox")
t

## Pointed Added

In [ ]:
df_e = fg.iloc[:,1:].nlargest(15,columns="Points_Added").reset_index(drop=True).reset_index()
df_e = df_e.drop(columns=["FGM","FGA","PTS","xPTS"])
df_e["index"] += 1
df_e["PLAYER_ID"] = df_e["PLAYER_ID"].astype(str)
# df_e["xPTS"] = df_e["xPTS"].round(1)

In [ ]:
t = (
    GT(df_e)
    .tab_header(
        title = md(f"**Best Volume Shot Makers {season_str}**"),
        subtitle = "Based on SSQM v2.0 | Shot Making: Points per shot (PPS) above league average"
    )
    .tab_source_note(source_note = "eFG%: Effective Field Goal % | xeFG%: Expected eFG% | xPTS: Expected Points")
    .tab_source_note(source_note = "Simple Shot Quality Model: SSQM v2.0 is based on shot type, defender distance & touch time" )
    .tab_source_note(source_note="bsky:@sradjoker.cc | X:@sradjoker | source: nba.com/stats")
    .cols_label(
        index = "#",
        PLAYER_ID = "",
        eFG="eFG%",
        xeFG="xeFG%",
        Shot_Making = "Shot Making",
        Points_Added = "Points Added",
    )
    .fmt_percent(
      columns=["eFG","xeFG"],
      decimals=1
    )
    .data_color(
        columns=['eFG','xeFG','Shot_Making'],
        palette="RdBu",
        reverse="False"
    )
    .fmt_image(
        columns="PLAYER_ID",
        path = img_DIR_P,
        file_pattern="{}.png"
    )
    .cols_align(align="center")
    .cols_align(align="left", columns="Player")
    .tab_options(
        heading_title_font_size="150%",
        heading_subtitle_font_size="110%",
        # heading_title_font_weight='bold',
        table_background_color="floralwhite",
        column_labels_font_size="105%",
        column_labels_font_weight='bold',
        row_group_font_weight='bold',
        row_group_background_color="#E5E1D8",
        table_font_size=10,
        table_font_names="Consolas", 
        data_row_padding = "2px",
        # table_margin_left = 7,
        # table_margin_right = 0,
    )
)
t.save(fig_DIR + f"ssqm2_points_added_{season}.png",scale=3,web_driver="firefox")
t

In [ ]:
df_e = fg.iloc[:,1:].query("PTS > 100").nsmallest(10,columns="Points_Added").reset_index(drop=True).reset_index()
df_e["index"] += 1
df_e["PLAYER_ID"] = df_e["PLAYER_ID"].astype(str)
df_e["xPTS"] = df_e["xPTS"].round(1)

In [ ]:
t = (
    GT(df_e)
    .tab_header(
        title = md(f"**Worst Volume Shot Makers {season_str}**"),
        subtitle = "Based on SSQM v2.0 | Shot Making: Points per shot (PPS) above league average"
    )
    .tab_source_note(source_note = "eFG%: Effective Field Goal % | xeFG%: Expected eFG% | xPTS: Expected Points")
    .tab_source_note(source_note = "Simple Shot Quality Model: SSQM v2.0 is based on shot type, defender distance & touch time" )
    .tab_source_note(source_note="bsky:@sradjoker.cc | X:@sradjoker | source: nba.com/stats")
    .cols_label(
        index = "#",
        PLAYER_ID = "",
        eFG="eFG%",
        xeFG="xeFG%",
        Shot_Making = "Shot Making",
        Points_Added = "Points Added",
    )
    .fmt_percent(
      columns=["eFG","xeFG"],
      decimals=1
    )
    .data_color(
        columns=['eFG','xeFG','Shot_Making'],
        palette="RdBu",
        reverse="False"
    )
    .fmt_image(
        columns="PLAYER_ID",
        path = img_DIR_P,
        file_pattern="{}.png"
    )
    .cols_align(align="center")
    .cols_align(align="left", columns="Player")
    .tab_options(
        heading_title_font_size="150%",
        heading_subtitle_font_size="110%",
        # heading_title_font_weight='bold',
        table_background_color="floralwhite",
        column_labels_font_size="105%",
        column_labels_font_weight='bold',
        row_group_font_weight='bold',
        row_group_background_color="#E5E1D8",
        table_font_size=10,
        table_font_names="Consolas", 
        data_row_padding = "2px",
        # table_margin_left = 7,
        # table_margin_right = 0,
    )
)
# t.save(fig_DIR + f"ssqm2_points_added_worst_{season}.png",scale=3,web_driver="firefox")
t

In [ ]:
df_e = fg.iloc[:,1:].query("PTS > 500").nsmallest(10,columns="xeFG").reset_index(drop=True)
# df_e = df_e.drop(columns=["Team"])
df_e.index += 1
df_e = df_e.reset_index()
df_e["PLAYER_ID"] = df_e["PLAYER_ID"].astype(str)
df_e["xPTS"] = df_e["xPTS"].round(1)

In [ ]:
t = (
    GT(df_e)
    .tab_header(
        title = md(f"**Toughest Shot Takers {season_str}**"),
        subtitle = "Based on SSQM v2.0 | Among Players Scoring at least 500 Pts" 
    )
    .tab_source_note(source_note = "eFG%: Effective Field Goal % | xeFG%: Expected eFG% | xPTS: Expected Points")
    .tab_source_note(source_note = "Simple Shot Quality Model: SSQM v2.0 is based on shot type, defender distance & touch time" )
    .tab_source_note(source_note="bsky:@sradjoker.cc | X:@sradjoker | source: nba.com/stats")
    .cols_label(
        index = "#",
        PLAYER_ID = "",
        eFG="eFG%",
        xeFG="xeFG%",
        Shot_Making = "Shot Making",
        Points_Added = "Points Added",
    )
    .fmt_percent(
      columns=["eFG","xeFG"],
      decimals=1
    )
    .data_color(
        columns=['eFG','xeFG','Shot_Making','Points_Added'],
        palette="RdBu",
        reverse="False"
    )
    .fmt_image(
        columns="PLAYER_ID",
        path = img_DIR_P,
        file_pattern="{}.png"
    )
    .cols_align(align="center")
    .cols_align(align="left", columns="Player")
    .tab_options(
        heading_title_font_size="150%",
        heading_subtitle_font_size="110%",
        # heading_title_font_weight='bold',
        table_background_color="floralwhite",
        column_labels_font_size="105%",
        column_labels_font_weight='bold',
        row_group_font_weight='bold',
        row_group_background_color="#E5E1D8",
        table_font_size=10,
        table_font_names="Consolas", 
        data_row_padding = "2px",
        # table_margin_left = 7,
        # table_margin_right = 0,
    )
)
# t.save(fig_DIR + f"ssqm2_toughest_{season}.png",scale=3,web_driver="firefox")
t

## Team

In [ ]:
df_e = fg.iloc[:,1:].query("PTS > 200").query("Team == 'Boston Celtics'").sort_values("Points_Added",ascending=False).reset_index(drop=True)
df_e = df_e.drop(columns=["Team","FGM","FGA","PTS","xPTS"])
df_e.index += 1
df_e = df_e.reset_index()
df_e["PLAYER_ID"] = df_e["PLAYER_ID"].astype(str)
# df_e["xPTS"] = df_e["xPTS"].round(1)

In [ ]:
t = (
    GT(df_e)
    .tab_header(
        title=md(f"Boston Celtics Shot Making {season_str}"),
        subtitle="Based on SSQM v2.0 | Shot Making: Points per shot (PPS) above league average"
    )
    .tab_source_note(source_note = "eFG%: Effective Field Goal % | xeFG%: Expected eFG% | xPTS: Expected Points")
    .tab_source_note(source_note = "Simple Shot Quality Model: SSQM v2.0 is based on shot type, defender distance & touch time" )
    .tab_source_note(source_note="bsky:@sradjoker.cc | X:@sradjoker | source: nba.com/stats")
    .cols_label(
        index = "#",
        PLAYER_ID = "",
        eFG="eFG%",
        xeFG="xeFG%",
        Shot_Making = "Shot Making",
        Points_Added = "Points Added",
    )
    .fmt_percent(
      columns=["eFG","xeFG"],
      decimals=1
    )
    .data_color(
        columns=['eFG','xeFG','Shot_Making'],
        palette="RdBu",
        reverse="False"
    )
    .fmt_image(
        columns="PLAYER_ID",
        path = img_DIR_P,
        file_pattern="{}.png"
    )
    .cols_align(align="center")
    .cols_align(align="left", columns="Player")
    .tab_options(
        heading_title_font_size="150%",
        heading_subtitle_font_size="110%",
        heading_title_font_weight='bold',
        table_background_color="floralwhite",
        column_labels_font_size="105%",
        column_labels_font_weight='bold',
        row_group_font_weight='bold',
        row_group_background_color="#E5E1D8",
        table_font_size=10,
        table_font_names="Consolas", 
        data_row_padding = "2px",
        # table_margin_left = 0,
        # table_margin_right = 0,
    )
)
t.save(fig_DIR + f"Celtics_Shot_Quality_{season}.png",scale=3,web_driver="firefox")
t

In [ ]:
xcvcxvx

In [ ]:
export_DIR = "../../repos/csv/"

In [ ]:
# df_e.to_csv(export_DIR + "NBA_Shot_Quality.csv")

In [ ]:
df_e.query("Player == 'Davion Mitchell'")

In [ ]:
df_e.sort_values("PTS")

In [ ]:
df_e.to_csv("NBA_Shot_Quality_V2.csv")

# To Website

In [ ]:
shots = pd.merge(df,df_avg,on=['general_range', 'closest_def', 'touch_time'])
shots["FG2_PCT"] = shots["FG2M"]/shots["FG2A"]
shots["FG3_PCT"] = shots["FG3M"]/shots["FG3A"]
shots = shots.replace([np.inf, -np.inf], np.nan)
shots = shots.fillna(0)
shots["PTS2"] =  (2*shots["FG2A"]*shots["FG2_PCT"]).round(2)
shots["PTS3"] =  (3*shots["FG3A"]*shots["FG3_PCT"]).round(2)
shots["PTS"] =  (2*shots["FG2A"]*shots["FG2_PCT"] + 3*shots["FG3A"]*shots["FG3_PCT"]).round(2)
shots["xPTS2"] = (2*shots["FG2A"]*shots["xFG2"]).round(2)
shots["xPTS3"] = (3*shots["FG3A"]*shots["xFG3"]).round(2)
shots["xPTS"] = (2*shots["FG2A"]*shots["xFG2"] + 3*shots["FG3A"]*shots["xFG3"]).round(2)

In [ ]:
fg = (
    shots
    .groupby(['PLAYER_ID'])[['FG2M', 'FG2A', 'FG3M', 'FG3A', 'FGM', 'FGA', 'PTS2', 'PTS3', 'PTS', 'xPTS2', 'xPTS3', 'xPTS']]
    .sum()
)
fg.columns = ['FG2M', 'FG2A', 'FG3M', 'FG3A','FGM', 'FGA', 'PTS2', 'PTS3', 'PTS', 'xPTS2', 'xPTS3', 'xPTS']
fg['FG2_PCT'] = np.round(fg['FG2M']/fg['FG2A'], 3)
fg['FG3_PCT'] = np.round(fg['FG3M']/fg['FG3A'], 3)
fg['eFG'] = np.round(fg['PTS']/fg['FGA']/2, 3)
fg['xeFG'] = np.round(fg['xPTS']/fg['FGA']/2, 3)
fg['Shot_Making2'] = np.round((fg['PTS2'] - fg['xPTS2'])/fg['FG2A'], 3)
fg['Shot_Making3'] = np.round((fg['PTS3'] - fg['xPTS3'])/fg['FG3A'], 3)
fg['Shot_Making'] = np.round((fg['PTS'] - fg['xPTS'])/fg['FGA'], 3)
fg = fg.drop(columns=['FGM', 'FGA'])
fg = fg.fillna(0)
fg = pd.merge(dfd,fg,on=["PLAYER_ID"])
fg["Points_Added2"] = fg['PTS2'] - fg['xPTS2']
fg["Points_Added3"] = fg['PTS3'] - fg['xPTS3']
fg["Points_Added"] = fg['PTS'] - fg['xPTS']
# fg["Points_Added"] = fg["Shot_Making"]* fg["FGA"]
fg["PTS"] = fg["PTS"].astype(int)
fg = fg.reset_index()
fg["Player"] = fg["PLAYER_ID"].map(player_dict)
fg.insert(1,"Player",fg.pop("Player"))
fg = pd.merge(fg,dft,on="PLAYER_ID")
fg["Team"] = fg["TEAM_ID"].map(teams_dict)
fg.insert(2,"Team",fg.pop("Team"))
fg[['Points_Added']] = fg[['Points_Added']].round(1)
fg[['Shot_Making']] = fg[['Shot_Making']].round(3)
fg = fg.drop(columns=["TEAM_ID"])
fg

In [ ]:
fg1 = fg.drop(columns=['index','FGM', 'FGA', 'FG2M', 'FG2A',
       'FG3M', 'FG3A',])
fg1 = fg1[['Player', 'Team', 'PLAYER_ID', 
            'PTS2', 'xPTS2', 'FG2_PCT', 'Shot_Making2', 'Points_Added2', 
            'PTS3', 'xPTS3', 'FG3_PCT', 'Shot_Making3', 'Points_Added3', 
            'PTS', 'xPTS', 'eFG', 'xeFG', 'Shot_Making', 'Points_Added'
       ]]
fg1[['xPTS2','Points_Added2','xPTS3','Points_Added3',]] = fg1[['xPTS2','Points_Added2','xPTS3','Points_Added3',]].round(2)
fg1[['xPTS','Points_Added',]] = fg1[['xPTS','Points_Added']].round(2)
fg1 = fg1.query("PTS >= 100").reset_index(drop=True)
fg1.to_csv(csv_export_DIR + "NBA_Shot_Quality_V2.csv")

In [ ]:
fg1.iloc[:30]

In [ ]:
csv_export_DIR